In [3]:
# 훈련 데이터 준비
import pandas as pd
from glob import glob
from pathlib import Path
from tqdm.auto import tqdm
import random
import shutil
import os

clips = glob('/home/kts123/aia/pwb/preprocess_PWB/preprocessed_pwb_2/*/*.pickle')
print(len(clips))
clips = [Path(e).parent for e in clips]
random.shuffle(clips)
clip_val = clips[:1]
clip_train = list(set(clips) - set(clip_val))
print(clip_train[0])
print(clip_val[0])
len(clip_train), len(clip_val)

402
/home/kts123/aia/pwb/preprocess_PWB/preprocessed_pwb_2/09_#2_thursday_03.05_30_000
/home/kts123/aia/pwb/preprocess_PWB/preprocessed_pwb_2/16_#2_thursday_05.00_30_000


(401, 1)

In [4]:
# data_root 폴더를 삭제하고 다시 만든다
shutil.rmtree('data_root', ignore_errors=True)
Path('data_root/train').mkdir(parents=True)
Path('data_root/val').mkdir(parents=True)

In [5]:
# link 걸기
def make_link(base, targets):
    for target in tqdm(targets):
        link = f'{base}/{Path(target).name}'
        cmd = f'ln -s "{target}" "{link}"'
        !$cmd
        
make_link('data_root/train',  clip_train)
make_link('data_root/val',    clip_val)

In [6]:
# fps 정보 저장(preprocess 에서 사연이 있어서 제대로 못 적은 경우 실행)
#!pip install imageio
#!pip install imageio-ffmpeg
mp4s = [e/'df_fan.pickle' for e in clips]

import imageio
for mp4 in tqdm(mp4s):
    fps= 30
    with open(mp4.parent/'fps.txt', 'w') as f:
        f.write(f'{fps:.2f}')

  0%|          | 0/394 [00:00<?, ?it/s]

In [4]:
# split 문제때문에 _no.jpg 로 바꿈
import os
from glob import glob
from tqdm.auto import tqdm

def rename_no(jpgs):
    for e in jpgs:
        no = e.replace('_yes.jpg', '_no.jpg') 
        os.rename(e, no)
        
clips = sorted(glob('data_root/train/*'))


# split 문제때문에 안전하게 start/end 12 프레임은 _no.jpg 로 바꿈
for e in tqdm(clips):
    jpgs = sorted(glob(f'{e}/*.jpg'))
    rename_no(jpgs[:12])
    rename_no(jpgs[-12:])

In [15]:
#split 문제 때문에 start/end 에서 움직임이 없는 프레임은 _no.jpg 로 바꿈
%pdb on
import cv2
import IPython.display as ipd
def rename_same(jpgs):
    prev = cv2.imread(jpgs[0])
    jpgs = jpgs[1:]
    same = []
    for e in jpgs:
        cur = cv2.imread(e)
        if not (prev == cur).all():
            break
        same.append(e)
    if len(same):
        same.append(jpgs[0])
    rename_no(same)
    return len(same)
        
    
sames = []
for e in tqdm(clips):
    jpgs = sorted(glob(f'{e}/*.jpg'))
    num_same_start = rename_same(jpgs)
    num_same_end = rename_same(jpgs[::-1])
    if 0 < num_same_start + num_same_end:
        sames.append([f'start:{num_same_start}, end:{num_same_end}', e])

Automatic pdb calling has been turned ON


In [16]:
sames

[['start:25, end:0', 'data_root/train/01_#1_thursday_02.00_30_000'],
 ['start:26, end:0', 'data_root/train/01_#1_thursday_02.01_00_000'],
 ['start:25, end:0', 'data_root/train/01_#2_thursday_02.00_30_000'],
 ['start:26, end:0', 'data_root/train/01_#2_thursday_02.01_00_000'],
 ['start:25, end:0', 'data_root/train/02_#1_thursday_01.00_30_000'],
 ['start:25, end:0', 'data_root/train/02_#2_thursday_01.00_30_000'],
 ['start:26, end:0', 'data_root/train/02_#2_thursday_01.01_00_000'],
 ['start:27, end:0', 'data_root/train/02_#2_thursday_01.01_30_000'],
 ['start:27, end:0', 'data_root/train/02_#2_thursday_01.02_00_000'],
 ['start:28, end:0', 'data_root/train/02_#2_thursday_01.02_30_000'],
 ['start:25, end:0', 'data_root/train/03_#1_thursday_01.00_30_000'],
 ['start:25, end:0', 'data_root/train/03_#2_thursday_01.00_30_000'],
 ['start:26, end:0', 'data_root/train/03_#2_thursday_01.01_00_000'],
 ['start:27, end:0', 'data_root/train/03_#2_thursday_01.01_30_000'],
 ['start:25, end:0', 'data_root/tr